## Read the data

In [10]:
import iris
import numpy

In [2]:
infile = "/g/data/ua6/drstree/CMIP5/GCM/CSIRO-BOM/ACCESS1-0/historical/mon/ocean/thetao/r1i1p1/thetao_Omon_ACCESS1-0_historical_r1i1p1_186501-186912.nc"

In [5]:
def lat_subset(cell):
    return -10 < cell < 10

def lon_subset(cell):
    return 30 < cell < 70

lat_constraint = iris.Constraint(latitude=lat_subset)
lon_constraint = iris.Constraint(longitude=lon_subset)

If I want to process the data on the native model grid, then you can't subset via lat and lon...

In [6]:
with iris.FUTURE.context(cell_datetime_objects=True):
    cube = iris.load_cube(infile, 'sea_water_potential_temperature' & lat_constraint & lon_constraint)

CoordinateMultiDimError: Cannot apply constraints to multidimensional coordinates

But by depth is fine...

In [7]:
level_subset = lambda cell: cell <= 50
level_constraint = iris.Constraint(depth=level_subset)

In [8]:
with iris.FUTURE.context(cell_datetime_objects=True):
    cube = iris.load_cube(infile, 'sea_water_potential_temperature' & level_constraint)

In [9]:
print cube

sea_water_potential_temperature          (time: 60; depth: 6; cell index along second dimension: 300; cell index along first dimension: 360)
     Dimension coordinates:
          time                                  x          -                                     -                                      -
          depth                                 -          x                                     -                                      -
          cell index along second dimension     -          -                                     x                                      -
          cell index along first dimension      -          -                                     -                                      x
     Auxiliary coordinates:
          latitude                            -          -                                     x                                      x
          longitude                           -          -                                     x                   

In [11]:
lev_bounds = cube.coord('depth').bounds
lev_diffs = numpy.apply_along_axis(lambda x: x[1] - x[0], 1, lev_bounds)

## Calculate something

In [13]:
def simple_integration(vert_vector, level_steps):
    """Perform vertical integration for a single lat/lon/time point."""
    
    areas = vert_vector * level_steps
    
    return areas.sum()

In [ ]:
integral = numpy.ma.apply_along_axis(simple_integration, 1, cube.data, lev_diffs)

## Rebuild the cube and write to file

[This page](http://esc24.github.io/iris/iris/iris/cube.html#iris.cube.Cube) tells you how.

In [ ]:
out_cube = iris.cube.Cube(integral,
                          standard_name='ocean_heat_content',
                          long_name='ocean_heat_content',
                          units=  ,
                          attributes= ,
                          dim_coords_and_dims=[(cube.coord('time'), 0), (cube.coord('latitude'), 1), (cube.coord('longitude'), 2)],
                          aux_coords_and_dims=
                         )

In [ ]:
# iris.save(my_cube, "myfile.nc")